# Part 1

--- Day 5: Binary Boarding ---

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like `FBFBBFFRLR`, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of `FBFBBFFRLR`:

```
Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
```

The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of `FBFBBFFRLR`:

```
Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.
```

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has `ID 44 * 8 + 5 = 357`.

Here are some other boarding passes:

```
BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.
```

As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

In [1]:
(import [pathlib [Path]])

In [2]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day05" "part1.txt"))

In [3]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [4]:
(defn row-column [seat]
  (, (cut seat 0 -3) (cut seat -3)))

In [5]:
(row-column "BFFFBBFRRR")

('BFFFBBF', 'RRR')

In [6]:
(defn rows-to-ints [rows]
  (tuple (gfor r rows (if (= r "F") 0 1))))

In [8]:
(defn cols-to-ints [cols]
  (tuple (gfor c cols (if (= c "L") 0 1))))

In [9]:
(defn halfway [upper lower]
  (+ lower (// (- upper lower) 2)))

In [37]:
(defn my-partition [range_ halves]
  (setv half (first halves))
  (setv [lower upper] range_)
  (if (= (- upper lower) 2)  ; only two options left
    (if (= half 0) lower (- upper 1))  ; upper is exclusive
    (if (= half 0)
      (my-partition (, lower (halfway lower upper)) (tuple (rest halves)))
      (my-partition (, (halfway lower upper) upper) (tuple (rest halves))))))

In [38]:
(my-partition (, 0 128) (rows-to-ints "BFFFBBF"))

70

In [39]:
(my-partition (, 0 128) (rows-to-ints "FBFBBFF"))

44

In [40]:
(defn seat-id [seat]
  (setv [row column] (row-column seat))
  (setv row-num (my-partition (, 0 128) (rows-to-ints row)))
  (setv col-num (my-partition (, 0 8) (cols-to-ints column)))
  (+ (* row-num 8) col-num))

In [41]:
(seat-id "BFFFBBFRRR")

567

In [42]:
(max (gfor seat (input-lines) (seat-id seat)))

888

# Part 2

Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

In [49]:
(defn by-two [seq]
  (yield-from (zip seq (islice seq 1 None))))

In [50]:
(first
  (gfor
    [id1 id2]
    (by-two (sorted (gfor seat (input-lines) (seat-id seat))))
    :if (= (- id2 id1) 2)
    (, id1 id2)))

(521, 523)